In [14]:
# https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772
# !pip install faiss-cpu
# !pip install sentence-transformers

In [15]:
import pandas as pd
data = [['Where are your headquarters located?', 'location'],
['Throw my cellphone in the water', 'random'], 
['Network Access Control?', 'networking'],
['Address', 'location']]
df = pd.DataFrame(data, columns = ['text', 'category'])

In [16]:
df

,text,category
0,Where are your headquarters located?,location
1,Throw my cellphone in the water,random
2,Network Access Control?,networking
3,Address,location


In [17]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

KeyboardInterrupt: 

In [5]:
type(vectors)

numpy.ndarray

In [6]:
vectors.shape

(4, 768)

In [7]:
vectors.shape[1]

768

In [8]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [9]:
import numpy as np

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [10]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [11]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [12]:
merge = pd.merge(results, df, left_on='ann', right_index=True)

In [13]:
merge

,distances,ann,text,category
0,0.584872,0,Where are your headquarters located?,location
1,1.175950,3,Address,location
2,1.644265,2,Network Access Control?,networking
3,1.919767,1,Throw my cellphone in the water,random
